In [1]:
!pip install python-dotenv openai langchain langchain_openai


     ------------------------------------- 327.5/327.5 kB 75.8 kB/s eta 0:00:00
     ------------------------------------ 975.5/975.5 kB 286.0 kB/s eta 0:00:00
     -------------------------------------- 45.9/45.9 kB 758.9 kB/s eta 0:00:00
     -------------------------------------- 75.6/75.6 kB 463.5 kB/s eta 0:00:00
     ------------------------------------ 423.1/423.1 kB 297.0 kB/s eta 0:00:00
     ------------------------------------ 332.8/332.8 kB 261.4 kB/s eta 0:00:00
     ------------------------------------ 127.5/127.5 kB 535.5 kB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 297.0 kB/s eta 0:00:00
     ------------------------------------ 798.9/798.9 kB 282.0 kB/s eta 0:00:00
     --------------------------------------- 77.9/77.9 kB 90.2 kB/s eta 0:00:00
     -------------------------------------- 58.3/58.3 kB 279.2 kB/s eta 0:00:00
  Using cached packaging-24.1-py3-none-any.whl (53 kB)
     ------------------------------------ 141.3/141.3 kB 195.3 kB

加载环境变量"OPENAI_API_KEY"

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from openai import  OpenAI
client = OpenAI(
    base_url = "https://api.moonshot.cn/v1"
)

In [9]:
completion = client.chat.completions.create(
    model = "moonshot-v1-128k",
    messages = [
        {'role': 'system','content' : "你被用于抑制用户的购买欲望。也就是说，当用户说想要买什么东西时，你需要提供合理的理由让用户不要购买。"},
        {'role': 'user', 'content': "我正在考虑购买一个键盘，但我想抑制购买欲望。你能帮我列出一些理由，让我思考一下我是否真的需要这个商品吗？"}
    ],
    max_tokens = 500,
    temperature = 0.6
)

print(completion.choices[0].message.content)

当然可以。以下是一些理由，帮助你评估是否真的需要购买一个新的键盘：

1. **现有键盘的使用寿命**：如果你的现有键盘仍然功能完好，没有出现故障或磨损，那么你可能不需要立即更换。

2. **财务状况**：考虑你的预算和当前的财务状况。如果购买新键盘会对你的经济状况造成压力，那么最好推迟购买。

3. **需求评估**：思考你是否真的需要一个新键盘。如果你只是出于想要拥有最新技术或设计的原因，而你的现有键盘仍然满足你的使用需求，那么购买新键盘可能不是必要的。

4. **技术升级的必要性**：评估新键盘是否提供了你的现有键盘所没有的功能，这些功能是否对你的工作或生活有实质性的改善。

5. **环境影响**：购买新商品会产生包装废物和可能的电子废物，这会对环境产生一定的影响。如果可能，考虑修理或重新利用现有的键盘。

6. **冲动购物**：问问自己这是否是一种冲动购物。有时候，我们只是想要一种新鲜感，而不是真正需要一个新键盘。

7. **替代方案**：考虑是否有其他更经济或更环保的方式来满足你对键盘的需求，比如清洁和维护现有的键盘，或者借用朋友的键盘。

8. **功能性与实用性**：新键盘是否具有你真正需要的特定功能？有时候，一些高端键盘的附加功能可能对大多数用户来说并不实用。

9. **技术过时的速度**：技术产品更新换代非常快，购买新键盘可能很快就会被更新的技术所超越。

10. **等待促销或折扣**：如果你确实需要一个新的键盘，考虑等待促销或折扣季节，这样可以节省一些开支。

通过这些理由，你可以更全面地评估是否真的需要购买一个新的键盘，以及这个购买决策是否明智。


In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system',"你被用于抑制用户的购买欲望。也就是说，当用户说想要买什么东西时，你需要提供合理的理由让用户不要购买。"),
        ('human',"我正在考虑购买一个{product}，但我想抑制购买欲望。你能帮我列出一些理由，让我思考一下我是否真的需要这个商品吗？")
    ]
)

prompt_template.format(product="iPhone 13")

'System: 你被用于抑制用户的购买欲望。也就是说，当用户说想要买什么东西时，你需要提供合理的理由让用户不要购买。\nHuman: 我正在考虑购买一个iPhone 13，但我想抑制购买欲望。你能帮我列出一些理由，让我思考一下我是否真的需要这个商品吗？'

In [13]:
model = ChatOpenAI(
    model = 'moonshot-v1-32k',
    base_url = "https://api.moonshot.cn/v1",
    max_tokens=500,
    temperature=0.6
)

In [16]:
def output_parser(output: str):
    parser_model = ChatOpenAI(
        model = 'moonshot-v1-32k',
        temperature=0.8,
        openai_api_base = "https://api.moonshot.cn/v1"
    )
    
    message = "你需要将传入的文本进行改写，尽可能自然且符合人类对话习惯而不是采用类似键值对的形式回答。这是你需要改写的文本:'{text}'"
    return parser_model.invoke(message.format(text = output))

注：理论上在第一步prompt的时候直接要求输出自然的文本也是可以的，但是在工程上为了实现稳定的结果，最好一步只做一件事。

In [17]:
# using langchain pipeline
chain = prompt_template | model | output_parser
answer = chain.invoke(input = {'product' : '耳机'})
print(answer.content)

如果你在考虑是否购买一副新耳机，这里有一些因素可能对你的决定有所帮助。首先，不要急于购买，检查一下你现有的耳机是否仍然功能完好。如果它们还处于良好状态，那么你或许没有必要立刻更换。

此外，我们需要考虑预算问题。买新耳机是否会给你带来财务压力？如果会，就等一等吧。科技产品更新换代非常快，你可能刚买了新款，不久就会有更先进的产品出现。所以，如果你不总是需要用耳机，或者只有在特定情况下才用，那么不必花大价钱购买。

我们还应该考虑音质。如果你对音质要求不高，现有的耳机可能已经足够满足你的需求。对环境的关注也很重要。购买新耳机会产生更多的电子垃圾。如果可能的话，考虑修理或升级你的现有耳机来减少环境影响。

如果你的生活方式即将发生变化，比如搬家、换工作或旅游，那么现在可能不是购买新耳机的最佳时机。而且，如果你不急于购买，可以等待促销活动，那时候可能会有更优惠的价格。

有时候，我们可能会考虑其他替代方案，比如使用音响系统或公共播放设备来满足我们的音频需求。购买耳机，是否是出于一时的冲动？还是经过深思熟虑的需求？最后，思考一下，购买新耳机是否与你的生活方式和价值观相符，比如你是否更倾向于简单生活或极简主义。

此外，你也可以考虑将预算用于其他方面，比如储蓄、投资、教育或健康，这可能会为你带来更大的长期回报。通过这些思考，你将能够更全面地评估自己是否真的需要购买新耳机，并决定何时购买最为合适。
